In [249]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp

from scipy.optimize import minimize
from scipy.integrate import solve_ivp
from scipy.interpolate import approximate_taylor_polynomial

In [250]:
from numpy import sqrt, sin, cos, log, pi, e

In [251]:
plt.style.use([
    'notebook',
    'grid',
    {'figure.dpi': 150}
])

In [252]:
# dados simulacao

r = .31
mu = .4
sigma = .55
a4 = .3019

In [253]:
# dSdt

def dSdt(_, s, u, a_1, a_2, a_3, b_1, b_2, b_3):
    h3, h4 = s

    if h3 < 0:
        h3 = 0
    
    q_out = (a_1 * sqrt(h3) + b_1) * 10e-5
    q_in = (a_2 * u + b_2) * 10e-5
    q_com = (a_3*(h4 - h3) + b_3) * 10e-5

    z = 2.5*np.pi*((h3-8)/100) - mu
    w = -((((h3-8)/100)-mu)**2)/(2*sigma**2)

    taylor_z = (1 - (z**2/2) + (z**4/24) - (z**6/720) + (z**8/40320))
    taylor_w = (1 + w + (w**2/2) + (w**3/6) + (w**4/24))

    a3 = (3*r/5)*(2.7*r - (1/(6*sqrt(2*pi)))*taylor_z*taylor_w)

    h3_dot = (q_com - q_out) / a3
    h4_dot = (q_in - q_com) / a4

    return [h3_dot, h4_dot]

In [254]:
def non_linear_space_state(_, s, u, a_1, a_2, a_3, b_1, b_2, b_3):
   h3, h4 = s

   if h3 < 0:
      h3 = 0

   z1 = np.sqrt(h3)
   z2 = np.cos(2.5*np.pi * (h4 - mu)) / (sigma * np.sqrt(2 * np.pi))
   z3 = np.exp(-((h4 - mu)**2) / (2 * sigma**2))

   a3 = ((3*r)/5) * (2.7*r - (z2 * z3))

   beta = (b_3 - b_1) / (a3 * h3)
   zeta = (b_2 - b_3) / (a4 * h4)

   A = np.array([
      [-(a_3/a3) -((a_1*z1)/(a3*h3)) + beta, (a_3/a3)],
      [(a_3/a4), -(a_3/a4) + zeta]
   ])

   B = np.array([
      [0],
      [a_2/a4]
   ])

   h = np.array([
      [h3],
      [h4]
   ])

   return 10e-5 * (A@h + B*u).flatten()

In [255]:
def simulate(s_0, coefficients, sim_time, sim_step, control_signal):
    time = np.arange(sim_time[0], sim_time[1], sim_step)
    time_span = [time[0], time[-1]]

    args = [control_signal]
    args = args + [coefficients[i] for i in range(len(coefficients))]

    s = solve_ivp(
        non_linear_space_state,
        y0=s_0,
        args=args,
        t_span=time_span,
        t_eval=time,
    )

    return s

In [256]:
c = np.array([165.48, 16.46, 29.4, -147.01, -156.93, -83.93])

sol = simulate(
    [1e-3, 1e-3],
    c,
    [0, 2500],
    1,
    42
)

In [257]:
sim_steps = np.array([
    1,
    1,
    1,
    1,
    1
])

sim_control_signals = np.array([42, 37, 28, 55, 47])

def multiple_inputs_simulation(coefficients):
    s_0 = [1e-3, 1e-3]
    sim_time = [0, 2500]

    solutions = []

    last_s = s_0
    last_t = sim_time

    for i in range(len(sim_steps)):
        s = simulate(
            last_s,
            coefficients,
            last_t,
            sim_steps[i], 
            sim_control_signals[i]
        )

        last_s = s.y[:,-1]
        last_t = [last_t[-1], last_t[-1] + 2500]

        solutions.append(s)


    return solutions

In [258]:
real_time = np.load('./data/t.npy')
level3 = np.load('./data/level3.npy')
level4 = np.load('./data/level4.npy')

# removing the noise using moving average
def moving_average(a, n=3):
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n


filtered_level3 = moving_average(level3, n=100)
filtered_level4 = moving_average(level4, n=100)

plt.plot(level3)
plt.plot(level4)

plt.plot(filtered_level3, label='h3')
plt.plot(filtered_level4, label='h4')
plt.legend()

In [259]:
sol = simulate(
    [1e-3, 1e-3],
    [165.48, 16.46, 29.4, -147.01, -156.93, -83.93],
    [0, 60],
    1,
    50
)

In [260]:
solutions = multiple_inputs_simulation(
    [165.48, 16.46, 29.4, -147.01, -156.93, -83.93]
)

y = np.hstack([s.y for s in solutions])
t = np.hstack([s.t for s in solutions])

h3 = y[0, :]
h4 = y[1, :]

In [261]:
t = t[:len(level3)]
h3 = h3[:len(level3)]
h4 = h4[:len(level3)]

In [262]:
%matplotlib qt
plt.plot(t, level4, label='h4 (real)')
plt.plot(t, h4, label='h4 (simulado)')

plt.plot(t, level3, label='h3 (real)')
plt.plot(t, h3, label='h3 (simulado)')

plt.legend()